In [1]:
%%time

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re


start_year = 2005

URL = 'https://www.resultados-futbol.com/ligue_1'
r = requests.get(URL)
page = r.content
soup = BeautifulSoup(page, 'html5lib')

current_year = int(soup.find('div', class_ = "titular-data").text.strip()[0:4])+1

end_year = current_year

Years = range(start_year,end_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

CPU times: user 1.47 s, sys: 131 ms, total: 1.6 s
Wall time: 2.02 s


In [2]:
table_fifa_all_years = pd.read_excel('Table_fifa_all_years.xlsx')
table_fifa_all_years = table_fifa_all_years.drop(['Unnamed: 0'], axis = 1)
table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,236,"['FC Porto', 'F.C. Porto']",Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,234,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,237,"['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo...",Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,1889,"['Belenenses', 'CF Os Belenenses', 'Belém', 'C...",Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,1891,"['Nacional', 'Clube Desportivo Nacional', 'CD ...",Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2022,France,Clermont Foot,72,70,72,71,/team/1815/clermont-foot/,1815,['Clermont Foot'],AS Saint-Étienne,4.5
2068,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,1,"['Arsenal', 'Arsenal FC']",Tottenham Hotspur,30.0
2069,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,17,['Southampton'],Portsmouth,7.5
2070,2014,Spain,Villarreal,75,74,72,74,/team/483/villarreal/fifa14/,483,"['Villarreal', 'Villarreal C.F.', 'Villarreal ...",Valencia Club de Fútbol,8.0


In [3]:
table_all_years = pd.read_excel('All_teams_results_leagues_countries_from_2005_edited.xlsx')
table_all_years = table_all_years.drop(['Unnamed: 0'], axis = 1)
table_all_years.head(3)

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team
0,2005,International,24 Feb 05,2005-02-24,Europa League,Europa League,Benfica,CSKA Moskva,1 - 1,1,1,x,1,1,4,1
1,2005,International,17 Feb 05,2005-02-17,Europa League,Europa League,CSKA Moskva,Benfica,2 - 0,2,0,1,3,0,0,3
2,2005,International,02 Dec 04,2004-12-02,Europa League,Europa League,KSK Beveren,Benfica,0 - 3,0,3,2,0,3,0,4


In [4]:
%%time

# create a dictionary whose keys are the team names in the original table
# and the values are the possible names in FIFA for each one of those teams

d = {}

# Build 2 lists: one with unique values of all teams in original table and
# another with unique values of all teams in fifa table

df1 = table_all_years[table_all_years['Competition']=='National League']

for year in Years:
    for country in Countries:
        original_teams_list_year_country = list(set(
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Home_team']) +\
            list(df1[(df1['Year'] == year) & 
                     (df1['Country'] == country)]
                     ['Away_team'])))

        fifa_teams_list_year_country = list(set(
            list(table_fifa_all_years[(table_fifa_all_years['Year'] == year) & 
                     (table_fifa_all_years['Country'] == country)]
                     ['Name'])))
        
        print(year)
        print(country)
        print('\n')

        
# Code to find a possible mismatch between the number of teams per year and
# per country in the original and fifa tables (of data is accurate, it should
# not happen)
        
        
        if len(original_teams_list_year_country) != len(fifa_teams_list_year_country):
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print(f'ERROR: {year},{country},\n\
            Number of teams original table: {len(original_teams_list_year_country)},\n\
            Number of teams fifa table: {len(fifa_teams_list_year_country)}')
            print('********************************************************')
            print('********************************************************')
            print('********************************************************')
            print('\n')
        
        else:

            # We create an auxiliar DataFrame (df) in order to sort the list of original
            # teams per year and per country by wuzzyfuzzy score, starting on the highest
            # score. This will allow to start matching names that have a higher level of
            # confidence, and consequently will allow to have a shorter list of names to
            # match for the last teams, whose level of confidence is lower.
            
                
            data = {'Original_name':original_teams_list_year_country}
            df = pd.DataFrame(data)
            
            df['Fifa_name_suggested'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[0])
    )
            
            df['Fifa_name_suggested_fuzzywuzzy_score'] = df['Original_name'].apply(
        (lambda x: process.extractOne(x, fifa_teams_list_year_country)[1])
    )
            
            df.sort_values(by = ['Fifa_name_suggested_fuzzywuzzy_score'], ascending = False,
                          inplace = True)
            
            original_teams_list_year_country = list(df['Original_name'])
            
  
            # Here we look for the best match between an original team name and a fifa team
            # name. For that we run different loops with different levels of fuzzywuzzy
            # score.
            
            
            fuzzywuzzy_score_level = [100,90,80,70,60,50,40,30,20,10,0]
            
            for original_team in original_teams_list_year_country:
                
                if original_team in d.keys():
                    continue
                
                else:

                    for score_level in fuzzywuzzy_score_level:

                        fuzzywuzzy_min_score = score_level

                        fifa_team_suggested = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[0]
                        fifa_team_suggested_score = process.extractOne(original_team, 
                                fifa_teams_list_year_country)[1]


                        if fifa_team_suggested_score >= fuzzywuzzy_min_score:
                            fifa_teams_list_year_country.remove(fifa_team_suggested)

                            if fifa_team_suggested not in d.values():
                                
                                
                                if original_team not in d.keys():
                                    d[original_team] = table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]['Fifa_team_all_names'].iloc[0]

                                else:
                                    d[original_team].append(
                                table_fifa_all_years.loc[table_fifa_all_years['Name']==fifa_team_suggested]
                                     ['Fifa_team_all_names']).iloc[0]
                                

                            print(original_team,'-->',fifa_team_suggested,'-->',fifa_team_suggested_score)

                            break
            
            print('\n')
            print('-----------------------------')
            print('\n')

2005
Portugal


Gil Vicente --> Gil Vicente --> 100
Benfica --> Benfica --> 100
Sporting Braga --> Sporting Braga --> 100
Boavista --> Boavista --> 100
Marítimo --> Marítimo --> 100
Moreirense --> Moreirense --> 100
Nacional --> Nacional --> 100
Rio Ave --> Rio Ave --> 100
União de Leiria --> União Leiria --> 95
Beira Mar SC --> SC Beira-Mar --> 95
Os Belenenses --> Belenenses --> 95
Estoril --> Estoril Praia --> 90
Penafiel --> F.C. Penafiel --> 90
Porto --> FC Porto --> 90
Vitória Guimarães --> Vitória SC --> 86
Vitória Setúbal --> Vitória Futebol Clube --> 86
Sporting CP --> Sporting Lisbon --> 80
Académica --> Coimbra --> 40


-----------------------------


2005
Spain


Real Betis --> Real Betis --> 100
Getafe --> Getafe --> 100
Real Madrid --> Real Madrid --> 100
Villarreal --> Villarreal --> 100
Real Zaragoza --> Real Zaragoza --> 100
Valencia --> Valencia --> 100
Levante --> Levante --> 100
Numancia --> Numancia --> 100
Osasuna --> Osasuna --> 100
Sevilla --> Sevilla --> 100
Al

Rayo Vallecano --> Rayo Vallecano de Madrid S.A.D. --> 90
Granada --> Granada Club de Fútbol --> 90


-----------------------------


2012
England


Queens Park Rangers --> Queens Park Rangers --> 100
Swansea City --> Swansea City --> 100


-----------------------------


2012
Italy


Novara --> Novara --> 100


-----------------------------


2012
Germany


FC Augsburg --> FC Augsburg --> 100


-----------------------------


2012
France


Dijon FCO --> Dijon FCO --> 100
Evian Thonon Gaillard --> Evian Thonon Gaillard FC --> 95


-----------------------------


2013
Portugal




-----------------------------


2013
Spain




-----------------------------


2013
England




-----------------------------


2013
Italy


Pescara --> Pescara --> 100


-----------------------------


2013
Germany


Fortuna Düsseldorf --> Fortuna Düsseldorf --> 100
Greuther Fürth --> SpVgg Greuther Fürth --> 95


-----------------------------


2013
France


Stade de Reims --> Stade de Reims --> 100


------

In [5]:
d

{'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'Sporting Braga': "['Sporting Braga', 'SC Braga', 'Braga']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Vitória Guimarães': "['Vitória SC', 'Vitória de Gui

In [6]:
# Manually correct the errors of the dictionary

print('Before manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

d['Milan'] = table_fifa_all_years[table_fifa_all_years['Name']=='AC Milan']\
['Fifa_team_all_names'].iloc[0]
d['Inter'] = table_fifa_all_years[table_fifa_all_years['Name']=='Inter Milan']\
['Fifa_team_all_names'].iloc[0]
d['Fiorentina'] = table_fifa_all_years[table_fifa_all_years['Name']=='Firenze']\
['Fifa_team_all_names'].iloc[0]
d['Köln'] = table_fifa_all_years[table_fifa_all_years['Name']=='FC Cologne']\
['Fifa_team_all_names'].iloc[0]
d['Feirense'] = table_fifa_all_years[table_fifa_all_years['Name']=='F. Santa Maria da Feira']\
['Fifa_team_all_names'].iloc[0]
d['Saint-Étienne'] = table_fifa_all_years[table_fifa_all_years['Name']=='AS Saint-Etienne']\
['Fifa_team_all_names'].iloc[0]
d['PSG'] = table_fifa_all_years[table_fifa_all_years['Name']=='Paris Saint-Germain']\
['Fifa_team_all_names'].iloc[0]

print('\n')
print('After manual correction:')
print('Milan:',d['Milan'])
print('Inter:',d['Inter'])
print('Fiorentina:',d['Fiorentina'])
print('Köln:',d['Köln'])
print('Feirense:',d['Feirense'])
print('Saint-Étienne:',d['Saint-Étienne'])
print('PSG:',d['PSG'])

Before manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['Hertha BSC Berlin', 'Hertha BSC', 'Hertha Berlin']
Feirense: ['Paços de Ferreira', 'Paços Ferreira', 'FC Paços de Ferreira']
Saint-Étienne: ['Paris Saint-Germain']
PSG: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']


After manual correction:
Milan: ['AC Milan', 'Milan']
Inter: ['Inter Milan', 'Inter']
Fiorentina: ['Firenze', 'Fiorentina', 'ACF Fiorentina']
Köln: ['FC Cologne', '1. FC Köln']
Feirense: ['F. Santa Maria da Feira', 'CD Feirense']
Saint-Étienne: ['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS Saint-Étienne']
PSG: ['Paris Saint-Germain']


In [7]:
d

{'Gil Vicente': "['Gil Vicente', 'V. Barcelos', 'Gil Vicente FC']",
 'Benfica': "['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']",
 'Sporting Braga': "['Sporting Braga', 'SC Braga', 'Braga']",
 'Boavista': "['Boavista', 'Boavista Futebol Clube', 'Boavista FC']",
 'Marítimo': "['Marítimo', 'Marítimo da Madeira', 'CS Marítimo', 'C. Funchal']",
 'Moreirense': "['Moreirense', 'Moreira de Cónegos', 'Moreirense FC']",
 'Nacional': "['Nacional', 'Clube Desportivo Nacional', 'CD Nacional', 'Funchal']",
 'Rio Ave': "['Rio Ave', 'Rio Ave FC']",
 'União de Leiria': "['União Leiria', 'União Desportivo de Leiria', 'União de Leiria, SAD']",
 'Beira Mar SC': "['SC Beira-Mar', 'SC Beira Mar']",
 'Os Belenenses': "['Belenenses', 'CF Os Belenenses', 'Belém', 'C.F. Os Belenenses', 'Os Belenenses']",
 'Estoril': "['Estoril Praia', 'Estoril', 'GD Estoril-Praia']",
 'Penafiel': "['F.C. Penafiel', 'FC Penafiel']",
 'Porto': "['FC Porto', 'F.C. Porto']",
 'Vitória Guimarães': "['Vitória SC', 'Vitória de Gui

In [8]:
# Add 2 auxiliar columns to df1 with all the fifa team names for both home
# and away team to help joining both tables

# Let's ignore the SettingWithCopyWarning, which in this case is a false positive
pd.set_option('mode.chained_assignment',None)

df1['Home_team_fifa_team_all_names'] = df1['Home_team'].apply(
        (lambda x: d[x])
    )

df1['Away_team_fifa_team_all_names'] = df1['Away_team'].apply(
        (lambda x: d[x])
    )

# Let's activate again the SettingWithCopyWarning
pd.reset_option("mode.chained_assignment")

df1

,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team,number_of_games_last_days_home_team,number_of_games_last_days_away_team,Home_team_fifa_team_all_names,Away_team_fifa_team_all_names
8,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,1 - 1,1,1,x,1,1,2,2,"['Boavista', 'Boavista Futebol Clube', 'Boavis...","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
9,2005,Portugal,14 May 05,2005-05-14,Liga Portuguesa,National League,Benfica,Sporting CP,1 - 0,1,0,1,3,0,3,5,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Sporting Lisbon', 'Sporting CP Lisbon', 'Spo..."
10,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,1 - 0,1,0,1,3,0,3,4,"['F.C. Penafiel', 'FC Penafiel']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
11,2005,Portugal,30 Apr 05,2005-04-30,Liga Portuguesa,National League,Benfica,Os Belenenses,1 - 0,1,0,1,3,0,4,3,"['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']","['Belenenses', 'CF Os Belenenses', 'Belém', 'C..."
12,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,1 - 2,1,2,2,0,3,3,4,"['Estoril Praia', 'Estoril', 'GD Estoril-Praia']","['Benfica', 'Sport Lisboa Benfica', 'SL Benfica']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54054,2022,France,18 Sep 21,2021-09-18,Ligue 1,National League,Saint-Étienne,Girondins Bordeaux,1 - 2,1,2,2,0,3,2,2,"['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS...","['Bordeaux', 'FC Girondins Bordeaux', 'FCGB', ..."
54055,2022,France,08 Aug 21,2021-08-08,Ligue 1,National League,Girondins Bordeaux,Clermont,0 - 2,0,2,2,0,3,0,0,"['Bordeaux', 'FC Girondins Bordeaux', 'FCGB', ...",['Clermont Foot']
54059,2022,France,07 Nov 21,2021-11-07,Ligue 1,National League,Saint-Étienne,Clermont,3 - 2,3,2,1,3,0,3,2,"['AS Saint-Etienne', 'A.S. Saint-Etienne', 'AS...",['Clermont Foot']
54060,2022,France,29 Aug 21,2021-08-29,Ligue 1,National League,Clermont,Metz,2 - 2,2,2,x,1,1,3,3,['Clermont Foot'],"['FC Metz', 'Football Club de Metz']"


In [9]:
%%time

# Join original table (df1) and fifa table

# first we start adding to the original table the columns we need
# from the fifa table just for the home teams

df_join_only_home = pd.DataFrame()

Columns = ['ATT','MID','DEF','OVR','Rival_team','Budget_Mill_€']


for year in Years:
    for country in Countries:
        df_1 = df1[(df1['Year'] == year) &
                  (df1['Country'] == country)
                  ]

        df2 = table_fifa_all_years

        df_2 = df2[(df2['Year'] == year) &
                  (df2['Country'] == country)
                  ]

        df_aux = pd.merge(df_1, df_2[['Fifa_team_all_names','ATT','MID','DEF','OVR','Rival_team','Budget_Mill_€']], 
            left_on= 'Home_team_fifa_team_all_names', 
            right_on = 'Fifa_team_all_names')

        df_join_only_home = pd.concat([df_join_only_home,df_aux])



df_join_only_home.drop(['Fifa_team_all_names'], axis=1, inplace=True)

for column in Columns:
    df_join_only_home.rename({
    column:f'Home_team_{column}'
    }, axis=1, inplace=True)


# Now that we have added the home team data, let's add it for the away team
    
df_join_total = pd.DataFrame()

for year in Years:
    for country in Countries:
        df_1 = df_join_only_home[(df_join_only_home['Year'] == year) &
                  (df_join_only_home['Country'] == country)
                  ]

        df2 = table_fifa_all_years

        df_2 = df2[(df2['Year'] == year) &
                  (df2['Country'] == country)
                  ]

        df_aux = pd.merge(df_1, df_2[['Fifa_team_all_names','ATT','MID','DEF','OVR','Rival_team','Budget_Mill_€']], 
            left_on= 'Away_team_fifa_team_all_names', 
            right_on = 'Fifa_team_all_names')

        df_join_total = pd.concat([df_join_total,df_aux])


df_join_total.drop(['Fifa_team_all_names'], axis=1, inplace=True)

for column in Columns:
    df_join_total.rename({
    column:f'Away_team_{column}'
    }, axis=1, inplace=True)

df_join_total

CPU times: user 4.8 s, sys: 451 ms, total: 5.25 s
Wall time: 5.33 s


,Year,Country,Date,Datetime_date_list,Competition_original_name_URL,Competition,Home_team,Away_team,Result,Home_score,...,Home_team_DEF,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€
0,2005,Portugal,22 May 05,2005-05-22,Liga Portuguesa,National League,Boavista,Benfica,1 - 1,1,...,53,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0
1,2005,Portugal,07 May 05,2005-05-07,Liga Portuguesa,National League,Penafiel,Benfica,1 - 0,1,...,55,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0
2,2005,Portugal,24 Apr 05,2005-04-24,Liga Portuguesa,National League,Estoril,Benfica,1 - 2,1,...,52,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0
3,2005,Portugal,10 Apr 05,2005-04-10,Liga Portuguesa,National League,Rio Ave,Benfica,1 - 0,1,...,48,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0
4,2005,Portugal,19 Mar 05,2005-03-19,Liga Portuguesa,National League,Vitória Setúbal,Benfica,0 - 2,0,...,56,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2022,France,29 Aug 21,2021-08-29,Ligue 1,National League,Troyes,Monaco,1 - 2,1,...,70,72,Stade de Reims,6.000,81,77,77,78,OGC Nice,26.0
165,2022,France,13 Aug 21,2021-08-13,Ligue 1,National League,Lorient,Monaco,1 - 0,1,...,71,72,Stade Rennais,7.000,81,77,77,78,OGC Nice,26.0
166,2022,France,01 Dec 21,2021-12-01,Ligue 1,National League,Angers SCO,Monaco,1 - 3,1,...,75,74,En Avant Guingamp,6.500,81,77,77,78,OGC Nice,26.0
167,2022,France,26 Sep 21,2021-09-26,Ligue 1,National League,Clermont,Monaco,1 - 3,1,...,72,71,AS Saint-Étienne,4.500,81,77,77,78,OGC Nice,26.0


In [10]:
# In order to simply the code from now on, and considering we
# already have all the info we need in just 1 table, let's call
# simply df to the new data frame

df = df_join_total

In [11]:
# Let's add another column that tells us if the game between the 2 teams
# is between rivals

df['Rivals'] = df.apply(
lambda x: 1 if (x['Home_team'] == x['Away_team_Rival_team'] or x['Away_team'] == x['Home_team_Rival_team']) else 0, axis = 1
)

# Let's drop some irrelevant columns from the df

df.drop(['Datetime_date_list','Competition_original_name_URL',
    'Home_score', 'Away_score',
    'Points_Home_Team', 'Points_Away_Team',
    'Home_team_fifa_team_all_names',
    'Away_team_fifa_team_all_names'], axis = 1, inplace = True)

df

,Year,Country,Date,Competition,Home_team,Away_team,Result,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
0,2005,Portugal,22 May 05,National League,Boavista,Benfica,1 - 1,x,2,2,...,58,Vitória SC,0.375,80,74,74,75,Sporting Lisbon,5.0,0
1,2005,Portugal,07 May 05,National League,Penafiel,Benfica,1 - 0,1,3,4,...,61,FC Porto,0.100,80,74,74,75,Sporting Lisbon,5.0,0
2,2005,Portugal,24 Apr 05,National League,Estoril,Benfica,1 - 2,2,3,4,...,59,Vitória Futebol Clube,2.000,80,74,74,75,Sporting Lisbon,5.0,0
3,2005,Portugal,10 Apr 05,National League,Rio Ave,Benfica,1 - 0,1,2,1,...,52,Benfica,2.900,80,74,74,75,Sporting Lisbon,5.0,1
4,2005,Portugal,19 Mar 05,National League,Vitória Setúbal,Benfica,0 - 2,2,4,4,...,61,Belenenses,0.350,80,74,74,75,Sporting Lisbon,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,2022,France,29 Aug 21,National League,Troyes,Monaco,1 - 2,2,2,5,...,72,Stade de Reims,6.000,81,77,77,78,OGC Nice,26.0,0
165,2022,France,13 Aug 21,National League,Lorient,Monaco,1 - 0,1,1,3,...,72,Stade Rennais,7.000,81,77,77,78,OGC Nice,26.0,0
166,2022,France,01 Dec 21,National League,Angers SCO,Monaco,1 - 3,2,2,3,...,74,En Avant Guingamp,6.500,81,77,77,78,OGC Nice,26.0,0
167,2022,France,26 Sep 21,National League,Clermont,Monaco,1 - 3,2,3,4,...,71,AS Saint-Étienne,4.500,81,77,77,78,OGC Nice,26.0,0


In [13]:
df.to_excel(
    'Merged_table_from_2005.xlsx')

In [14]:
df['Country'].value_counts()

Spain       6617
England     6609
Italy       6591
France      6528
Germany     5328
Portugal    4789
Name: Country, dtype: int64

In [15]:
for country in Countries:
    for year in Years:
        df_ = df[(df['Year'] == year) & (df['Country'] == country)]
        print(year, df_['Country'].value_counts())
    print('\n')

2005 Portugal    306
Name: Country, dtype: int64
2006 Portugal    306
Name: Country, dtype: int64
2007 Portugal    240
Name: Country, dtype: int64
2008 Portugal    240
Name: Country, dtype: int64
2009 Portugal    240
Name: Country, dtype: int64
2010 Portugal    240
Name: Country, dtype: int64
2011 Portugal    240
Name: Country, dtype: int64
2012 Portugal    240
Name: Country, dtype: int64
2013 Portugal    240
Name: Country, dtype: int64
2014 Portugal    240
Name: Country, dtype: int64
2015 Portugal    306
Name: Country, dtype: int64
2016 Portugal    306
Name: Country, dtype: int64
2017 Portugal    306
Name: Country, dtype: int64
2018 Portugal    306
Name: Country, dtype: int64
2019 Portugal    306
Name: Country, dtype: int64
2020 Portugal    306
Name: Country, dtype: int64
2021 Portugal    306
Name: Country, dtype: int64
2022 Portugal    115
Name: Country, dtype: int64


2005 Spain    380
Name: Country, dtype: int64
2006 Spain    380
Name: Country, dtype: int64
2007 Spain    380
Name: 

In [16]:
year = 2005
country = 'Italy'

df_ = df[(df['Year'] == year) & (df['Country'] == country)]
print(df_['Home_team'].value_counts(),df_['Away_team'].value_counts())

Parma         20
Bologna       20
Brescia       19
Lecce         19
Juventus      19
Udinese       19
Lazio         19
Cagliari      19
Inter         19
Siena         19
Milan         19
Chievo        19
Fiorentina    19
Reggina       19
Palermo FC    19
Atalanta      19
Sampdoria     19
Livorno       19
Messina       19
Roma          19
Name: Home_team, dtype: int64 Parma         20
Bologna       20
Juventus      19
Siena         19
Udinese       19
Lazio         19
Cagliari      19
Inter         19
Milan         19
Lecce         19
Reggina       19
Brescia       19
Palermo FC    19
Atalanta      19
Sampdoria     19
Livorno       19
Messina       19
Chievo        19
Fiorentina    19
Roma          19
Name: Away_team, dtype: int64


In [17]:
team = 'Parma'

df_[(df_['Home_team'] == team) | (df_['Away_team'] == team)]

,Year,Country,Date,Competition,Home_team,Away_team,Result,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
10,2005,Italy,06 Mar 05,National League,Parma,Cagliari,3 - 2,1,5,3,...,74,Modena,5.0,71,63,61,64,Palermo,1.0,0
19,2005,Italy,15 May 05,National League,Juventus,Parma,2 - 0,1,3,4,...,87,Inter Milan,70.0,76,71,70,74,Modena,5.0,0
20,2005,Italy,12 Dec 04,National League,Livorno,Parma,2 - 0,1,2,3,...,64,Firenze,1.0,76,71,70,74,Modena,5.0,0
21,2005,Italy,23 Apr 05,National League,Milan,Parma,3 - 0,1,5,5,...,91,Inter Milan,80.0,76,71,70,74,Modena,5.0,0
22,2005,Italy,27 Feb 05,National League,Lazio,Parma,2 - 0,1,3,5,...,77,AS Roma,18.0,76,71,70,74,Modena,5.0,0
23,2005,Italy,13 Feb 05,National League,Fiorentina,Parma,2 - 1,1,5,4,...,79,Juventus,1.0,76,71,70,74,Modena,5.0,0
24,2005,Italy,17 Apr 05,National League,Chievo,Parma,2 - 0,1,1,3,...,67,Parma,1.0,76,71,70,74,Modena,5.0,1
25,2005,Italy,19 Dec 04,National League,Roma,Parma,5 - 1,1,3,4,...,79,Lazio,50.0,76,71,70,74,Modena,5.0,0
26,2005,Italy,23 Jan 05,National League,Messina,Parma,1 - 0,1,4,4,...,59,Reggina Calcio,1.0,76,71,70,74,Modena,5.0,0
27,2005,Italy,07 Nov 04,National League,Palermo FC,Parma,1 - 1,x,4,6,...,66,Catania,1.0,76,71,70,74,Modena,5.0,0


In [18]:
year = 2013
country = 'England'

df_ = df[(df['Year'] == year) & (df['Country'] == country)]
print(df_['Home_team'].value_counts(),df_['Away_team'].value_counts())

Queens Park Rangers     19
Wigan Athletic          19
Southampton             19
Man. City               19
Man. Utd                19
Everton                 19
West Ham                19
Arsenal                 19
Reading                 19
Newcastle               19
Sunderland              19
Tottenham Hotspur       19
Aston Villa             19
Chelsea                 19
Norwich City            19
Swansea City            19
Liverpool               19
West Bromwich Albion    19
Fulham                  19
Stoke City              19
Name: Home_team, dtype: int64 Norwich City            19
Wigan Athletic          19
Arsenal                 19
Southampton             19
Man. City               19
Everton                 19
Queens Park Rangers     19
West Ham                19
Man. Utd                19
Reading                 19
Sunderland              19
Tottenham Hotspur       19
Aston Villa             19
Chelsea                 19
Newcastle               19
Swansea City            1

In [19]:
team = 'Man. Utd'

df_[(df_['Home_team'] == team) | (df_['Away_team'] == team)]

,Year,Country,Date,Competition,Home_team,Away_team,Result,1x2,number_of_games_last_days_home_team,number_of_games_last_days_away_team,...,Home_team_OVR,Home_team_Rival_team,Home_team_Budget_Mill_€,Away_team_ATT,Away_team_MID,Away_team_DEF,Away_team_OVR,Away_team_Rival_team,Away_team_Budget_Mill_€,Rivals
0,2013,England,19 May 13,National League,West Bromwich Albion,Man. Utd,5 - 5,x,3,3,...,74,Wolverhampton Wanderers,8.0,86,81,82,82,Manchester City,30.0,0
1,2013,England,28 Apr 13,National League,Arsenal,Man. Utd,1 - 1,x,3,4,...,80,Tottenham Hotspur,24.0,86,81,82,82,Manchester City,30.0,0
2,2013,England,17 Apr 13,National League,West Ham,Man. Utd,2 - 2,x,3,4,...,74,Tottenham Hotspur,7.5,86,81,82,82,Manchester City,30.0,0
3,2013,England,14 Apr 13,National League,Stoke City,Man. Utd,0 - 2,2,2,3,...,75,West Bromwich Albion,15.0,86,81,82,82,Manchester City,30.0,0
4,2013,England,30 Mar 13,National League,Sunderland,Man. Utd,0 - 1,2,2,2,...,75,Newcastle United,16.0,86,81,82,82,Manchester City,30.0,0
5,2013,England,23 Feb 13,National League,Queens Park Rangers,Man. Utd,0 - 2,2,2,4,...,75,Fulham,10.0,86,81,82,82,Manchester City,30.0,0
6,2013,England,02 Feb 13,National League,Fulham,Man. Utd,0 - 1,2,5,5,...,77,Chelsea,13.0,86,81,82,82,Manchester City,30.0,0
7,2013,England,20 Jan 13,National League,Tottenham Hotspur,Man. Utd,1 - 1,x,3,4,...,80,Arsenal,22.0,86,81,82,82,Manchester City,30.0,0
8,2013,England,01 Jan 13,National League,Wigan Athletic,Man. Utd,0 - 4,2,4,4,...,74,Bolton Wanderers,8.0,86,81,82,82,Manchester City,30.0,0
9,2013,England,23 Dec 12,National League,Swansea City,Man. Utd,1 - 1,x,3,3,...,74,Cardiff City,7.0,86,81,82,82,Manchester City,30.0,0
